# Groq

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/groq.ipynb)

Le code ci-après requiert un compte avec Groq. Groq se distingue par un temps d'inférence extrêmement rapide.

Docs:
- https://github.com/groq/groq-python
- https://groq.com/

In [2]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)
# !pip3 install -q -U scikit-learn

# À installer dans tous les cas pour Google Colab et Github
if ('google.colab' in sys.modules) or ('CI' in os.environ):
    !pip3 install -q -U groq
    !pip3 install -q -U groqflow

## Chargement de la Clé pour Groq

Il vous faut obtenir de Groq une clé pour exécuter ce bloc-note Jupyter. Vous pouvez consulter [Getting Started](https://console.groq.com/docs/quickstart). Ensuite, le chargement se fait soit à partir de l'environnement (fichier `.env`), soit à partir des secrets de Google Colab.

In [4]:
groq_api_key = None
if 'google.colab' in sys.modules:
  from google.colab import userdata
  groq_api_key = userdata.get('GROQ_API_KEY')
else:
  from dotenv import load_dotenv, find_dotenv
  _ = load_dotenv(find_dotenv()) # lire le fichier .env local
  groq_api_key = os.environ['GROQ_API_KEY']

In [5]:
from groq import Groq

client = Groq(api_key=groq_api_key)

## Conversations Chat

In [7]:
MODEL = 'llama3-70b-8192' 
# MODEL = 'mixtral-8x7b-32768'

messages = [
  { 'role': 'system', 'content': 'Tu es un professeur d\'histoire.' },
  { 'role': 'user', 'content': 'Quand la première guerre mondiale a-t-elle commencé?' },
  { 'role': 'assistant', 'content': 'La première guerre mondiale a commencé en 1914.' },
  { 'role': 'user', 'content': 'Quand a-t-elle fini?' }
]

completion = client.chat.completions.create(
    messages=messages,
    model=MODEL  
)

response = completion.choices[0].message.content
print(response)

La Première Guerre mondiale a pris fin le 11 novembre 1918, lorsque l'armistice a été signé entre les Alliés et l'Allemagne. Cependant, le traité de Versailles, qui a officiellement mis fin à la guerre, n'a été signé que le 28 juin 1919.


## Groqflow

En attente de création de compte sur https://support.groq.com/#/register, suite à la signature d'un NDA.

```
StageError: groq-devtools>=0.9.2.1 is a required dependency for this part of GroqFlow. However, groq-devtools was not found. Please contact sales@groq.com to get access to groq-devtools.
```